# Importar bibliotecas

In [118]:
import pandas as pd
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)# Removendo notações científicas
import requests, json
from Conexão_Banco_de_Dados_Oracle import conectar_bd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
np.set_printoptions(suppress=False) # Removendo notações científicas
from pycep_correios import get_address_from_cep, WebService
import math
from datetime import datetime

# Remover warnings dos kernels
import warnings
warnings.filterwarnings('ignore')

# Conectar ao banco de dados (cursor)

In [119]:
conn = conectar_bd()
cursor = conn.cursor()

# Extrair informações dos clientes via banco de dados

In [120]:
# Dados cadastrais
cons_empresas_cadastradas = '''
SELECT 
    a.CODEMPRESA, a.RAZAOSOCIAL, a.NOMEFANTASIA, 
    d.CEP, d.NUMCNPJCPF
FROM HINOERP_BIO.GEEMPRESA a
    LEFT JOIN HINOERP_BIO.GEEMPRESAPARAMESTAB b
        ON a.CODEMPRESA = b.CODEMPRESA
    LEFT JOIN HINOERP_BIO.GEENDEMPRESA c
        ON a.CODEMPRESA = c.CODEMPRESA
    LEFT JOIN HINOERP_BIO.GEENDERECO d
        ON c.CODENDERECO = d.CODENDERECO
    WHERE TIPOEMPRESA = 'C' AND LENGTH(d.NUMCNPJCPF) = 14
'''
df_empresas_cadastradas = pd.read_sql(cons_empresas_cadastradas, conn)

# Retornando data de cadastro das empresas
cons = '''
SELECT e.DATALOG as DATACADASTRO, PARA AS CODEMPRESA
FROM HINOERP_BIO.GELOG E
LEFT JOIN HINOERP_BIO.GELOGDET b
ON E.CODLOG = b.CODLOG
WHERE TABELA = 'GEEMPRESA' AND CAMPO = 'Cód. Empresa' AND DE IS NULL
'''
datas_cadastros = pd.read_sql(cons, conn)
datas_cadastros['CODEMPRESA'] = datas_cadastros['CODEMPRESA'].astype(int)

# Merge das informações
df_empresas_cadastradas = df_empresas_cadastradas.merge(datas_cadastros, on='CODEMPRESA', how='left')

# Dados financeiros
cons_dados_financeiros = '''
SELECT 
    e.CODEMPRESA, 
    SUM(e.VALORORIGINAL) AS TOTALVENDAS, 
    AVG(e.VALORJUROS) AS MEDIAJUROS, 
    AVG(e.DATALIQUIDACAO - e.DATAVENCIMENTO) AS DIASATRASO
FROM HINOERP_BIO.FITITULOS e
WHERE e.INDICENFSAIDA IS NOT NULL
GROUP BY CODEMPRESA
'''

df_titulos = pd.read_sql(cons_dados_financeiros, conn)

# Unindo as informações
df = df_empresas_cadastradas.merge(df_titulos, on='CODEMPRESA', how='left').fillna(0)
df.index = [i for i in range(df.shape[0])]

# Extrair informações extras dos clientes via request

In [121]:
# Função de extração de dados do CNPJ
def _dados_cnpj(cnpj):
    return json.loads(requests.get(f'https://thecollector.linkana.com/companies?cnpj=eq.{cnpj}&limit=1%27').text)[0]

# Buscando informações de geolocalização
def busca_geolocation(cep):
    try:
        # Ajustar para parametro de pesquisa URL
        '''
        Obs: Foi acrescentado o replace para Bonfim Paulista pois era uma exceção que estava ocasionando erros "desenecessários"
        '''
        r = lambda x: x.replace('Bonfim Paulista (Ribeirão Preto)', 'Ribeirão Preto').replace(' ', '+')

        URL = "https://nominatim.openstreetmap.org/search?format=json&addressdetails=1&country=$brazil&state=${}&city=${}&limit=1"
        infos = get_address_from_cep(cep, webservice=WebService.CORREIOS)
        dados_cep = json.loads(requests.get(URL.format(infos['uf'], r(infos['cidade']))).text)[0]

        return (dados_cep['lat'], dados_cep['lon'], dados_cep['importance'], dados_cep['address']['state_district'])
    except:
        return (np.nan, np.nan, np.nan, np.nan)

In [122]:
# Teste função de geolocalização
busca_geolocation('44025930')

(nan, nan, nan, nan)

In [ ]:
# Criação de um dicionário auxiliar
dict_aux = {
    'Data Abertura CNPJ': [],
    'Capital Social': [],
    'Natureza Jurídica': [],
    'Porte': [],
    "CNAE's": [],
    "Latitude": [],
    "Longitude": [],
    "NvlImportanciaGEO": [],
    "Região": []
}

# Alimentando o dicionário auxiliar
for i, cnpj in enumerate(df['NUMCNPJCPF']):
    dados_consulta_cnpj = _dados_cnpj(cnpj)
    lat, lon, importance, regiao = busca_geolocation(dados_consulta_cnpj['cep'])
    
    # Dados geolocalização
    dict_aux['Latitude'].append(lat)
    dict_aux['Longitude'].append(lon)
    dict_aux['NvlImportanciaGEO'].append(importance)
    dict_aux['Região'].append(regiao)
    
    # Dados CNPJ
    dict_aux['Data Abertura CNPJ'].append(dados_consulta_cnpj['data_inicio_atividade'])
    dict_aux['Capital Social'].append(str(dados_consulta_cnpj['capital_social'])[:-2])
    dict_aux['Natureza Jurídica'].append(dados_consulta_cnpj['natureza_juridica'])
    dict_aux['Porte'].append(dados_consulta_cnpj['cod_porte_empresa'])
    cnaes = [dados_consulta_cnpj['cnae_fiscal']]
    if dados_consulta_cnpj['cnaes_secundarios'] != None:
        cnaes = cnaes + dados_consulta_cnpj['cnaes_secundarios']
    dict_aux["CNAE's"].append(cnaes)
    

    print(f'{i + 1} / {df.shape[0] } - {df["RAZAOSOCIAL"][i]} - {regiao}')

# Transformando o DF auxiliar em objeto Pandas e concatenando com o DF de infos baixadas
df_aux = pd.DataFrame.from_dict(dict_aux)

df = pd.concat([df, df_aux], axis=1)

1 / 451 - GETMED - DISTRIBUIDORA MEDICO HOSPITALAR LTDA - Região Geográfica Intermediária de Bauru
2 / 451 - ORION COMERCIO DE PRODUTOS MEDICOS LTDA - Região Geográfica Intermediária de Belo Horizonte
3 / 451 - OSTEO SOLUTION - COMERCIO, IMPORTACAO E EXPORTACAO DE ARTIGO - Região Geográfica Intermediária de São Paulo
4 / 451 - PATMUS PRODUTOS MEDICOS E HOSPITALARES LTDA - Região Geográfica Intermediária do Rio de Janeiro
5 / 451 - RADIOMED COMERCIO E IMPORTACAO LTDA - Região Geográfica Intermediária de São Paulo
6 / 451 - SURGICAL TEC COMERCIO DE PRODUTOS HOSPITALARES LTDA - Região Geográfica Intermediária de Cuiabá
7 / 451 - 4 IDEA COMERCIO, IMPORTACAO E EXPORTACAO DE PRODUTOS MEDICOS - Região Geográfica Intermediária de Campinas
8 / 451 - LIFE SPINE IMPLANTES LTDA - Região Geográfica Intermediária de São Paulo
9 / 451 - LIGHT CARE COMERCIO DE PRODUTOS MEDICOS E HOSPITALARES EIREL - Região Geográfica Intermediária de São Paulo
10 / 451 - ORTOMUNDI COMERCIO DE MATERIAIS MEDICO CIRURGIC

80 / 451 - OCTOMED COMERCIO DE PRODUTOS MEDICOS LTDA - Região Geográfica Intermediária de São Paulo
81 / 451 - PEGURIN COMERCIO DE PRODUTOS HOSPITALARES LTDA EPP - Região Geográfica Intermediária de Campos dos Goytacazes
82 / 451 - SPINE LEVEL COM. IMP.E LOC. PROD. MEDICOS EIRELI - Região Geográfica Intermediária de São Paulo
83 / 451 - TITANIUN COM DE MAT MED HOSP E SERV LTDA ME - Região Geográfica Intermediária de Cuiabá
84 / 451 - TORIDE INDUSTRIA E COMERCIO LTDA - Região Geográfica Intermediária de Campinas
85 / 451 - DG INOX COMERCIO DE METAIS LTDA - ME - Região Geográfica Intermediária de Campinas
86 / 451 - MEDICAL PRIME IMP. EXP. E DISTR DE PROD MED. HOSP. - Região Geográfica Intermediária de São Paulo
87 / 451 - MEDIC PRODUTOS CIRURGICOS EIRELI - Região Geográfica Intermediária de Belo Horizonte
88 / 451 - NOREM PRODUTOS MEDICOS E HOSPITALARES LTDA - Região Geográfica Intermediária de Campo Grande
89 / 451 - MARIO MEDICAL COMERCIO DE MATERIAL HOSPITALAR LTDA - Região Geográfic

# Pré-tratamento do DF

In [ ]:
# Preenchendo a feature  de data de cadastro (clientes que foram cadastrados na importação)
df['DATACADASTRO'] = df['DATACADASTRO'].fillna('2020-01-01T00:00:00')

In [ ]:
# Dummies de CNAE's
df = df.explode("CNAE's")

In [ ]:
dist_cnaes = df["CNAE's"].value_counts()
sns.barplot(dist_cnaes[:20].values, dist_cnaes[:20].index)
print("Percentual  de CNAE's atendidos: ", sum(dist_cnaes[:20] / df.shape[0]))

Devido ao grande número de CNAE's, decidi por reduzir a dimensionalidade destes valores antes de tratar o DF mais a frente, mantendo apenas os top 20 CNAE's mais frequentes das empresas cadastradas.

In [ ]:
df = df.loc[df["CNAE's"].isin(dist_cnaes[:20].index)]

dummies = pd.get_dummies(df["CNAE's"])
df = df.drop("CNAE's", axis=1)
df = pd.concat([df, dummies], axis=1)
df = df[~df.index.duplicated(keep='first')]
df_backup = df.copy()

# I - EDA - Análise Exploratória de Dados

In [ ]:
df.head()

In [ ]:
df.tail()

## I.I - Análise estrutural

In [ ]:
rows_df = df.shape[0]
cols_df = df.shape[1]
print(f'Quantidade total de linhas: {rows_df}\nQuantidade total de colunas: {cols_df}')

In [ ]:
df.info()

### Reajustando datatypes

In [ ]:
df['Data Abertura CNPJ'] = pd.to_datetime(df['Data Abertura CNPJ'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['DATACADASTRO'] = pd.to_datetime(df['DATACADASTRO'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [ ]:
df['Capital Social'].value_counts()

In [ ]:
df['Capital Social'] = df['Capital Social'].replace('', 0).astype(float)
df['Porte'] = df['Porte'].astype(int)
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

### Removendo colunas indesejadas
Colunas de identificação, localização (redundante).

In [ ]:
df = df.drop(['CODEMPRESA', 'RAZAOSOCIAL', 'NOMEFANTASIA', 'CEP', 'NUMCNPJCPF'], axis=1)

## I.II - Análise Qualitativa

### Features categóricas

In [ ]:
for feature_nn in df.dtypes[df.dtypes == object].index:
    print(feature_nn.center(100, '*'))
    print(f'{(sum(df[feature_nn].isna()) / rows_df) * 100:.2f}% das features são missing.')
    print(f'Dos {sum(~df[feature_nn].isna())}, sendo distribuidos em {len(df[feature_nn].value_counts())} valores diferentes.')

In [ ]:
top_regioes = (df['Região'].value_counts() / rows_df)[df['Região'].value_counts() / rows_df > 0.02]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(top_regioes, top_regioes.index)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

In [ ]:
# Substituir regiões com baixa densidade de empresas cadastradas por "outra" (exceto NaN's).
df['Região'][(~df['Região'].isin(top_regioes.index)) & (~df['Região'].isna())] = 'Outra'

In [ ]:
# Análise de portes das empresas
sns.barplot(df['Porte'].value_counts().index, df['Porte'].value_counts())

In [ ]:
# Análise de correlação entre as duas features categóricas
analise_ft_cat = pd.concat([pd.get_dummies(df['Porte']), df['Região']], axis=1)
analise_ft_cat = analise_ft_cat.groupby(['Região'], as_index=False)[[1,3,5]].sum()
analise_ft_cat['Totais'] = analise_ft_cat[[1,3,5]].sum(axis=1)

df_exp = analise_ft_cat[[1,3,5]]
somas_cols = df_exp.sum(axis=0)
for col in df_exp:
    percent = (somas_cols[somas_cols.index == col] /somas_cols.sum()).values[0]
    df_exp[col] = analise_ft_cat['Totais'] * percent
chi2 = (((df_exp - analise_ft_cat[df_exp.columns])**2) / df_exp).sum().sum()
pearson_coef = math.sqrt(chi2/(chi2 + analise_ft_cat['Totais'].sum())) * math.sqrt(3/(3-1))
print(f'O nível de associação, de acordo com o coeficiente de contingência de Pearson é de {pearson_coef * 100:.2f}%')

Podemos observar que a relação entre as features são moderadamente leves.

In [ ]:
# Encoding das features categóricas (exceto porte, pois se trata de uma variável categórica ordinal)

In [ ]:
dummies_cat = pd.get_dummies(df[['Região', 'Natureza Jurídica']])
df = df.drop(['Região', 'Natureza Jurídica'], axis=1)
df = pd.concat([df, dummies_cat], axis=1)

### Features númericas

<font color=red>Tratando a feature de data de abertura do CNPJ. Decidi por tratar como tempo em atividade, onde optamos por um período anual.<font color=red>

In [ ]:
df['AnosAtividade'] = datetime.now() - df['Data Abertura CNPJ']
df['AnosAtividade'] = np.round(df['AnosAtividade'].dt.days / 365, 2)
df = df.drop('Data Abertura CNPJ', axis=1)

<font color=red>Criando nova feature para substituir a feature de total de vendas. Isto porque apenas o total de vendas não nos dá muitas informações sobre o cliente estar ou não ativo, ou ter ou não um bom volume de compras. Por isto, decidi por alterar esta feature por uma média<font color=red>

In [ ]:
df['MédiaVendasMês'] = datetime.now() - df['DATACADASTRO']
df['MédiaVendasMês'] = np.round(df['TOTALVENDAS'] / (df['MédiaVendasMês'].dt.days / 30), 2)
df = df.drop(['DATACADASTRO'], axis=1)

- Correlações

In [ ]:
numericals = ['TOTALVENDAS', 'MEDIAJUROS', 'DIASATRASO', 'Capital Social', 
              'NvlImportanciaGEO', 'MédiaVendasMês', 'AnosAtividade']
sns.heatmap(df[numericals].corr(), annot=True, vmin=-1, cmap='coolwarm_r')

<font color=red>Como esperado, a feature de MédiaVendasMês e Total Vendas estão fortemente correlacionadas, sendo assim, iremos remover a coluna de TotalVendas. Também houve uma correlação moderada entre as colunas de DiasAtraso e MediaJuros, porém, optei por mantê-las por enquanto.<font color=red>

In [ ]:
df = df.drop('TOTALVENDAS', axis=1)
numericals.pop(0)

- Resumo dos dados

In [ ]:
for feature_nn in df[numericals]:
    print(feature_nn.center(100, '*'))
    print(f'Tipo da variável: {df[feature_nn].dtype}')
    print(f'{(sum(df[feature_nn].isna()) / rows_df) * 100:.2f}% das features são missing.')
    print(f'Os {sum(~df[feature_nn].isna())} dados, estão distribuidos em {len(df[feature_nn].value_counts())} valores diferentes.\n')

- Distribuição dos dados

In [ ]:
df[numericals].describe()

In [ ]:
fig, ax = plt.subplots(len(numericals), 2, figsize=(12, 36))

for i, col in enumerate(df[numericals].columns):
    axes_hist = ax[i][0]
    axes_box  = ax[i][1]
    sns.histplot(data=df, x=col, ax=axes_hist)
    sns.boxplot(x=col, data=df, ax=axes_box)

<font color=red>Alguns Boxplots ficaram concentrados em escalas curtas (próximas de zero). Daremos uma olhada mais afundo à seguir.<font color=red>

In [ ]:
for i, col in enumerate(df[numericals].columns):
    zeros = df[col].loc[df[col] > 0].count()
    print(f'A coluna {col} tem {zeros} ({zeros/df[col].shape[0]*100:.2f}%) amostras diferentes de zero.')

- Tratamento de missing values

In [ ]:
print('Percentual de features vazias:')
percent_missing = (df.isnull().sum() / df.shape[0]) * 100
percent_missing

<font color=red>Como tivemos poucas amostras missing, optarei por remover as mesmas e prosseguir com o PCA e modelo.<font color=red>

In [ ]:
# print(f'Quantidade de linhas pré-drop: {df.shape[0]}.')
# df = df.dropna()
# print(f'Quantidade de linhas pós-drop: {df.shape[0]}.')